## 🦜🔗 LangChain Agent mit einem SQL Toolkit


Zuerst enpacken wir unsere Demo Datenbank mit der wir gleich arbeiten werden.


In [1]:
! [ ! -f northwind.db ] && bunzip2 -k northwind.db.bz2

#### Wir erstellen das SQL Toolkit und den Agenten


In [2]:
from langchain_community.agent_toolkits.sql.base import SQLDatabaseToolkit
from langchain_community.utilities.sql_database import SQLDatabase
from helpers import llm

model = llm(temperature=0)
db = SQLDatabase.from_uri("sqlite:///northwind.db")
toolkit = SQLDatabaseToolkit(db=db, llm=model)
tools = toolkit.get_tools()

In [10]:
from langgraph.prebuilt import create_react_agent
from langchain.schema import AIMessage, SystemMessage
from langchain_core.runnables import chain

system_message_prompt = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.
DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.
If the question does not seem related to the database, just return "I don't know" as the answer."""


@chain
def messages_modifier(messages):
    return [
        SystemMessage(system_message_prompt.format(dialect=toolkit.dialect, top_k=10)),
        messages[0],
        AIMessage(
            "I should look at the tables in the database to see what I can query. Then I should query the schema of the most relevant tables."
        ),
        *messages[1:],
    ]


agent_executor = create_react_agent(model, tools, messages_modifier=messages_modifier)

/var/folders/01/ywnzdvn56c16b5vdy9c7_q8w0000gn/T/ipykernel_26226/764382512.py:29: LangGraphDeprecationWarning: Parameter 'messages_modifier' in function 'create_react_agent' is deprecated as of version 0.1.9 and will be removed in version 0.3.0. Use 'state_modifier' parameter instead.
  agent_executor = create_react_agent(model, tools, messages_modifier=messages_modifier)


#### Looos....


In [11]:
from langchain.schema import HumanMessage


input_1 = {"messages": [HumanMessage(content="Where do I find the orders?")]}
print(input_1["messages"][0])
for chunk in agent_executor.stream(input_1):
    for state in chunk.values():
        for message in state["messages"]:
            message.pretty_print()

content='Where do I find the orders?'
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_kM3FBr5gSC5OoJAAB38cvgdj)
 Call ID: call_kM3FBr5gSC5OoJAAB38cvgdj
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

Categories, CustomerCustomerDemo, CustomerDemographics, Customers, EmployeeTerritories, Employees, Order Details, Orders, Products, Regions, Shippers, Suppliers, Territories
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_fqn2Pm86yfC64djp6BuhTOEt)
 Call ID: call_fqn2Pm86yfC64djp6BuhTOEt
  Args:
    table_names: Orders
  sql_db_schema (call_qAR28p6YiextvjeWgbstFcgt)
 Call ID: call_qAR28p6YiextvjeWgbstFcgt
  Args:
    table_names: Order Details
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE "Orders" (
	"OrderID" INTEGER N

In [12]:
input_2 = {"messages": [HumanMessage(content="Which Employee has the most orders?")]}
for chunk in agent_executor.stream(input_2):
    for state in chunk.values():
        for message in state["messages"]:
            message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_dQ2luOY8cChPXasFlMe53WCt)
 Call ID: call_dQ2luOY8cChPXasFlMe53WCt
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

Categories, CustomerCustomerDemo, CustomerDemographics, Customers, EmployeeTerritories, Employees, Order Details, Orders, Products, Regions, Shippers, Suppliers, Territories
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_W2KnH8SeWJ8Z5FNhU7vffbTj)
 Call ID: call_W2KnH8SeWJ8Z5FNhU7vffbTj
  Args:
    table_names: Employees
  sql_db_schema (call_7ov1Xw4espSyrHT8bk8gvEC5)
 Call ID: call_7ov1Xw4espSyrHT8bk8gvEC5
  Args:
    table_names: Orders
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE "Employees" (
	"EmployeeID" INTEGER, 
	"LastName" TEXT, 
	"FirstName" TEX

#### Schauen wir mal, ob er das hier hinbekommt.


In [15]:
input_3 = {
    "messages": [
        HumanMessage(
            content="Which Customer has had the Order with the highest total cost ever? What was the Order Id?"
        )
    ]
}
for chunk in agent_executor.stream(input_3):
    for state in chunk.values():
        for message in state["messages"]:
            message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_WRo7jPSzKBNq2EdZUXEAegGY)
 Call ID: call_WRo7jPSzKBNq2EdZUXEAegGY
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

Categories, CustomerCustomerDemo, CustomerDemographics, Customers, EmployeeTerritories, Employees, Order Details, Orders, Products, Regions, Shippers, Suppliers, Territories
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_rpxVjrQENOZXG5HeW82ezGPQ)
 Call ID: call_rpxVjrQENOZXG5HeW82ezGPQ
  Args:
    table_names: Orders
  sql_db_schema (call_PdbKeotFfwvpmk2hd1suoJLH)
 Call ID: call_PdbKeotFfwvpmk2hd1suoJLH
  Args:
    table_names: Customers
  sql_db_schema (call_Y7WMWoAWI9cw2rCo2FXSlLO5)
 Call ID: call_Y7WMWoAWI9cw2rCo2FXSlLO5
  Args:
    table_names: Order Details
================================= Tool Message ===============

#### Und noch einmal das batchen (async) demonstrieren.


In [13]:
from typing import Dict


async def format_output(item: Dict) -> str:
    return [item.get("messages")[0].content, item.get("messages")[-1].content]


batcher = agent_executor | format_output

In [16]:
result = await batcher.abatch([input_1, input_2, input_3])


for index, item in enumerate(result):
    print(f"Query {index+1}:")
    print(f"Question: {item[0]}")
    print(f"Answer: {item[1]}\n\n")

Query 1:
Question: Where do I find the orders?
Answer: Here are the most recent orders:

1. **Order ID:** 15271
   - **Customer ID:** EASTC
   - **Order Date:** 2023-09-13
   - **Ship Name:** Maison Dewey
   - **Ship City:** Bruxelles
   - **Ship Country:** Belgium

2. **Order ID:** 17398
   - **Customer ID:** KOENE
   - **Order Date:** 2023-09-12
   - **Ship Name:** Simons bistro
   - **Ship City:** Kobenhavn
   - **Ship Country:** Denmark

3. **Order ID:** 13810
   - **Customer ID:** ROMEY
   - **Order Date:** 2023-09-12
   - **Ship Name:** Wilman Kala
   - **Ship City:** Helsinki
   - **Ship Country:** Finland

4. **Order ID:** 26065
   - **Customer ID:** PARIS
   - **Order Date:** 2023-09-11
   - **Ship Name:** Around the Horn
   - **Ship City:** Colchester
   - **Ship Country:** UK

5. **Order ID:** 15220
   - **Customer ID:** BERGS
   - **Order Date:** 2023-09-11
   - **Ship Name:** Ana Trujillo Emparedados y helados
   - **Ship City:** México D.F.
   - **Ship Country:** Mexico



## ✅ Aufgabe

#### Ändert alle Namen, die ihr in der Datenbank findet auf "Little Bobbie Tables;" (https://xkcd.com/327/)


In [ ]:
print(db.get_table_info())

In [ ]:
your_input = {"messages": [HumanMessage(content="")]}
for chunk in agent_executor.stream(your_input):
    for state in chunk.values():
        for message in state["messages"]:
            message.pretty_print()